### Inspect adversarial dataset

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [168]:
model_folder = os.getenv("RESNET_MODEL_FOLDER")
print(model_folder)

target_class = -1

In [169]:
from nadf.data.adversarial import load_or_create_dataset

dataset = load_or_create_dataset(
    folder=model_folder,
    target_class=target_class,
    num_attacks_eps_coef=[(4, 0.25), (2, 0.5), (3, 1.0), (1, 2.0)],
    recreate=False,
    verbose=False,
)

In [170]:
dataset

In [171]:
dataset.keys()

In [172]:
dataset["y"]["train"].unique()
dataset["y"]["val"].unique()
dataset["y"]["test"].unique()


### Make regression dataset

In [173]:
from nadf.data.datasets import create_regression_dataset

In [198]:
from torch.utils.data import Dataset
import torch


regression_datasets = {}
clean_upweight_factor = 3.0


# def create_regression_dataset(
#     z_all,
#     y_all,
#     z_clean,
#     y_clean,
#     distance_metric="euclidean",
#     clean_upweight_factor=1.0,
#     z_clean_pool=None,
#     y_clean_pool=None,
#     split="train",

for split in ["train", "val", "test"]: 
    y_adv_for_matching = torch.full((len(dataset["y_adv"][split]),), target_class)  # Override to all 9s # Use 9s, not true labels
    print(y_adv_for_matching)
    y_adv = y_adv_for_matching
    
    if target_class == -1:
        
        y_adv = dataset["y_adv"][split]
        print(y_adv)
    # regression_datasets[split] = create_regression_dataset(
    #     z_all = dataset["z"][split],
    #     y_all = y_all_for_matching,
    #     z_clean = dataset["z_clean"][split],
    #     y_clean = dataset["y_clean"][split],
    #     z_clean_pool = dataset["z_clean_pool"][split],
    #     y_clean_pool = dataset["y_clean_pool"][split])

    regression_datasets[split] = create_regression_dataset(
        z_clean = dataset["z_clean"][split],
        y_clean = dataset["y_clean"][split],
        z_adv = dataset["z_adv"][split],
        #y_adv = dataset["y_adv"][split],
        y_adv=y_adv,  
        clean_upweight_factor=clean_upweight_factor,
    )

In [199]:
import torch
import os

# After creating the regression dataset
regression_dataset_path = os.path.join(model_folder, "adversarial_examples", str(target_class), "regression_dataset.pt")

# Save the dataset
os.makedirs(os.path.dirname(regression_dataset_path), exist_ok=True)
torch.save(regression_datasets, regression_dataset_path)
print(f"Saved regression dataset to {regression_dataset_path}")

In [200]:
# anchors, distances, labels, upweights

# check for unique values in distances
regression_datasets["train"][1].unique()

## train probe

In [201]:
import torch
import os
import dotenv
from argparse import Namespace
from nadf.training.pipeline import train_probe_model
from typing import Dict, Any
import os, glob, torch
from nadf.training.pipeline import load_probe_model


dotenv.load_dotenv()
model_folder = os.getenv("RESNET_MODEL_FOLDER")
print(model_folder)

In [202]:
regression_dataset_path = os.path.join(model_folder, "adversarial_examples", str(target_class), "regression_dataset.pt")
regression_datasets = torch.load(regression_dataset_path)


In [203]:
# DEBUG

# Check for target distances = 0
exact_zeros = regression_datasets["train"][1] == 0
num_exact_zeros = (regression_datasets["train"][1] == 0).sum()

num_exact_zeros

In [204]:
# anchors, distances, labels, upweights
regression_datasets["train"][1]

In [205]:
datasets = {}

for split in ["train", "val", "test"]: 
    anchors, distances, labels, sample_weights = regression_datasets[split]
    # anchors, distances, labels, sample_weights = create_regression_dataset(
    #     z_all = dataset["z"][split],
    #     y_all = torch.full((len(dataset["y"][split]),), 9),  # All 9s for matching # ! NOT SURE THIS IS VALID
    #     z_clean = dataset["z_clean"][split],
    #     y_clean = dataset["y_clean"][split],
    #     z_clean_pool = dataset["z_clean_pool"][split],
    #     y_clean_pool = dataset["y_clean_pool"][split])
    
    # Convert tuple to dictionary
    datasets[split] = {
        "anchors": anchors,
        "distances": distances,
        "labels": labels,
        "sample_weights": sample_weights
    }


input_dim = anchors.shape[1]
print(f"Input dimension: {input_dim}")
# Now you can train
args = Namespace(
    input_dim=input_dim, 
    depth=5, 
    width=1024, 
    activation="relu",
    loss="huber",
    batch_size=256,
    model_type="mlp",
    epochs=30,
    lr=0.001,
    weight_decay=0.0001,
    verbose=True,
    num_epochs=30,  
    huber_delta=1.0,  
    checkpoint_name=None,  # Auto-generate
    save_dir="./checkpoints",
    upweight=clean_upweight_factor,  # Clean upweight factor (1.0 = no upweighting)
    augmentation="none",  # No augmentation
    num_augmentations=0,  # Number of augmentations
    target_class=target_class,
)

# Create the save directory
os.makedirs(args.save_dir, exist_ok=True)


checkpoint_path, final_metrics = train_probe_model(datasets, args)


## evaluate probe

In [206]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy.stats import pearsonr, spearmanr
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score




In [207]:
# checkpoint_path = "./checkpoints/mlp_d3w512_mse_up3.0x.pth"
checkpoint_path = checkpoint_path
checkpoint = torch.load(checkpoint_path, map_location='cuda' if torch.cuda.is_available() else 'cpu')


# Check what's in the checkpoint
print("Checkpoint keys:", checkpoint.keys())
print("\nModel config keys:", checkpoint['model_config'].keys())
print("\nTraining config keys:", checkpoint['training_config'].keys())

# Print some key values
print("\n" + "="*60)
print("CHECKPOINT DETAILS")
print("="*60)


print(f"Total epochs trained: {checkpoint['training_config']['num_epochs']}")
print(f"Test Loss: {checkpoint['training_config']['final_test_loss']:.4f}")
print(f"Val Loss: {checkpoint['training_config']['final_val_loss']:.4f}")

# Model architecture
print(f"\nModel: {checkpoint['model_config']['model_type']}")
print(f"Depth: {checkpoint['model_config']['depth']}, Width: {checkpoint['model_config']['width']}")
print(f"Activation: {checkpoint['model_config']['activation']}")
print(f"Loss function: {checkpoint['training_config']['loss_function']}")


In [208]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model, checkpoint = load_probe_model(checkpoint_path, device)
model.eval()
print(f"Loaded model from: {checkpoint_path}")

In [209]:
# Build test loader → compute predictions/targets → plot

# 1) Build test loader from datasets dict
test_loader = DataLoader(
    TensorDataset(
        datasets["test"]["anchors"],
        datasets["test"]["distances"],
        datasets["test"]["labels"],
        datasets["test"]["sample_weights"],
    ),
    batch_size=256,
    shuffle=False,
)

# 2) Inference to get predictions/targets
device = next(model.parameters()).device  # assumes 'model' is already defined
all_predictions, all_targets = [], []
with torch.no_grad():
    for anchors, distances, labels, weights in test_loader:
        preds = model(anchors.to(device)).cpu().squeeze()
        all_predictions.append(preds)
        all_targets.append(distances.squeeze())

all_predictions = torch.cat(all_predictions)
all_targets = torch.cat(all_targets)

# Masks
clean_mask = all_targets == 0
adv_mask = all_targets > 0

# 3) Scatter: Predicted vs True
plt.figure(figsize=(7, 6))
plt.scatter(all_targets[clean_mask], all_predictions[clean_mask], alpha=0.3, s=10, label='Clean', color='tab:blue')
plt.scatter(all_targets[adv_mask],   all_predictions[adv_mask],   alpha=0.3, s=10, label='Adversarial', color='tab:red')
mx = float(all_targets.max())
plt.plot([0, mx], [0, mx], 'k--', lw=1, label='Perfect prediction')
plt.xlabel('True Distance'); plt.ylabel('Predicted Distance'); plt.title('Predicted vs True Distances')
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()

# 4) Histogram: Overlaid True vs Predicted (shared bins)
true_all = all_targets.cpu().numpy().astype(float)
pred_all = all_predictions.cpu().numpy().astype(float)
xmin, xmax = float(np.min(true_all)), float(np.max(true_all)); xmax = xmax if xmax > xmin else xmin + 1e-6
edges = np.linspace(xmin, xmax, 51)

plt.figure(figsize=(7, 6))
plt.hist(true_all, bins=edges, alpha=0.6, density=True, label='True', color='tab:blue')
plt.hist(pred_all, bins=edges, alpha=0.6, density=True, label='Predicted', color='tab:orange')
plt.xlabel('Distance'); plt.ylabel('Density'); plt.title('Distribution of True vs Predicted Distances')
plt.legend(); plt.grid(True, alpha=0.3); plt.tight_layout(); plt.show()



# Metrics: correlation, tolerance accuracy (±0.1/±0.2), and binary classification at threshold
# Ensure 1D CPU numpy arrays
true = all_targets.view(-1).cpu().numpy().astype(float)
pred = all_predictions.view(-1).cpu().numpy().astype(float)

# 1) Correlations
pearson_r, pearson_p = pearsonr(true, pred)
spearman_r, spearman_p = spearmanr(true, pred)
# Optional R^2
ss_res = np.sum((true - pred) ** 2)
ss_tot = np.sum((true - true.mean()) ** 2) + 1e-12
r2 = 1.0 - ss_res / ss_tot

print("="*60)
print("CORRELATIONS")
print(f"Pearson r:  {pearson_r:.4f} (p={pearson_p:.2e})")
print(f"Spearman r: {spearman_r:.4f} (p={spearman_p:.2e})")
print(f"R^2:        {r2:.4f}")

# 2) AUROC 
adv_labels = (true > 0).astype(int)  # 1 for adversarial, 0 for clean
roc_auc = roc_auc_score(adv_labels, pred)  # Higher distance = more adversarial
print(f"\nAUROC (adversarial as positive): {roc_auc:.4f}")
print("="*60)

# 2) Tolerance accuracy (within ±eps of true)
def within_tolerance_acc(true_arr, pred_arr, eps):
    return float(np.mean(np.abs(pred_arr - true_arr) <= eps))

acc_01 = within_tolerance_acc(true, pred, 0.10)
acc_02 = within_tolerance_acc(true, pred, 0.20)

print("\nTOLERANCE ACCURACY")
print(f"Within ±0.10: {acc_01:.4f}")
print(f"Within ±0.20: {acc_02:.4f}")

# Also show per-group (clean vs adv) tolerance accuracy
clean_mask = (all_targets == 0).cpu().numpy()
adv_mask   = (all_targets > 0).cpu().numpy()

if clean_mask.any():
    acc_01_clean = within_tolerance_acc(true[clean_mask], pred[clean_mask], 0.10)
    acc_02_clean = within_tolerance_acc(true[clean_mask], pred[clean_mask], 0.20)
    print(f"Within ±0.10 (clean): {acc_01_clean:.4f}")
    print(f"Within ±0.20 (clean): {acc_02_clean:.4f}")

if adv_mask.any():
    acc_01_adv = within_tolerance_acc(true[adv_mask], pred[adv_mask], 0.10)
    acc_02_adv = within_tolerance_acc(true[adv_mask], pred[adv_mask], 0.20)
    print(f"Within ±0.10 (adv):   {acc_01_adv:.4f}")
    print(f"Within ±0.20 (adv):   {acc_02_adv:.4f}")

# 3) Binary classification with threshold on predicted distance
#    Predict "clean" when pred < threshold; true clean when true == 0
threshold = 0.10  # change as needed
pred_labels = (pred < threshold).astype(int)
true_labels = (true == 0).astype(int)

acc = (pred_labels == true_labels).mean()
tp = int(((pred_labels == 1) & (true_labels == 1)).sum())
fp = int(((pred_labels == 1) & (true_labels == 0)).sum())
tn = int(((pred_labels == 0) & (true_labels == 0)).sum())
fn = int(((pred_labels == 0) & (true_labels == 1)).sum())

precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
f1        = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

print("\nBINARY CLASSIFICATION (predict clean if pred < threshold)")
print(f"Threshold: {threshold}")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1:        {f1:.4f}")
print(f"Confusion Matrix: TP={tp}  FP={fp}  FN={fn}  TN={tn}")
print("="*60)

## Save results

In [210]:
model, checkpoint = load_probe_model(checkpoint_path, device)

print(checkpoint['model_config'])
print(checkpoint['training_config'])

In [211]:
# === Save results to CSV (create/append and back up existing) ===
import os
import shutil
from pathlib import Path
from datetime import datetime
import pandas as pd

# Absolute paths as requested
results_dir = Path('/rds/general/user/nk1924/home/NADF/data/results')
results_dir.mkdir(parents=True, exist_ok=True)
csv_path = results_dir / 'metrics.csv'
model, checkpoint = load_probe_model(checkpoint_path, device)


# Try to gather relevant hyperparameters from globals/model
g = globals()
def _first(*vals):
    for v in vals:
        if v is not None:
            return v
    return None

# Safely check for model without causing NameError
width = checkpoint['model_config']['width']
depth = checkpoint['model_config']['depth']
upweight = checkpoint['training_config']['upweight_factor']

model_class = checkpoint['model_config']['model_type']

lr = checkpoint['training_config']['learning_rate']
weight_decay = checkpoint['training_config']['weight_decay']
batch_size_used = checkpoint['training_config']['batch_size']
epochs = checkpoint['training_config']['num_epochs']
loss_function = checkpoint['training_config']['loss_function']

# Try to get losses from multiple possible sources
train_loss = None
val_loss = None
test_loss = None

# Source 1: final_metrics dictionary (from train_regression_model return)
if 'final_metrics' in g:
    train_loss = g['final_metrics'].get('final_train_loss')
    val_loss = g['final_metrics'].get('final_val_loss')
    test_loss = g['final_metrics'].get('final_test_loss')

# Source 2: checkpoint['training_config']
if train_loss is None and 'checkpoint' in g:
    tc = g['checkpoint'].get('training_config', {})
    train_loss = _first(tc.get('final_train_loss'), tc.get('train_loss'))
    val_loss = _first(tc.get('final_val_loss'), tc.get('val_loss'), tc.get('best_val_loss'))
    test_loss = _first(tc.get('final_test_loss'), tc.get('test_loss'))

# Source 3: Individual global variables
if train_loss is None:
    train_loss = _first(g.get('train_loss'), g.get('final_train_loss'))
if val_loss is None:
    val_loss = _first(g.get('val_loss'), g.get('final_val_loss'), g.get('best_val_loss'))
if test_loss is None:
    test_loss = _first(g.get('test_loss'), g.get('final_test_loss'))

# Safely get true and pred arrays from globals
if 'true' in g and 'pred' in g:
    true_arr = g['true']
    pred_arr = g['pred']
elif 'all_targets' in g and 'all_predictions' in g:
    true_arr = g['all_targets'].view(-1).cpu().numpy().astype(float)
    pred_arr = g['all_predictions'].view(-1).cpu().numpy().astype(float)
else:
    raise NameError("Neither 'true'/'pred' nor 'all_targets'/'all_predictions' are defined. Please run the metrics section first.")

# Class distribution (ground-truth): clean (true==0) vs adversarial (true>0)
n_clean = int((true_arr == 0).sum())
n_adv = int((true_arr > 0).sum())

# Get n_samples
n_samples = len(true_arr)

# Safely get metrics from globals (they should exist if true/pred exist)
if 'pearson_r' not in g or 'acc_01' not in g:
    raise NameError("Metrics variables (pearson_r, acc_01, etc.) not found. Please run the metrics section first.")

row = {
    'timestamp': datetime.now().isoformat(timespec='seconds'),
    'model_class': model_class,
    'width': width,
    'depth': depth,
    'loss_function': loss_function,
    'upweight': upweight,
    'lr': lr,
    'weight_decay': weight_decay,
    'batch_size': batch_size_used,
    'epochs': epochs,
    'n_samples': int(n_samples),

    # target class info (integer)
    'target_class': target_class,  # positive class used in metrics (true == 0 → labeled 1)
    'n_clean': n_clean,
    'n_adv': n_adv,

    # losses
    'train_loss': float(train_loss) if train_loss is not None else None,
    'val_loss': float(val_loss) if val_loss is not None else None,
    'test_loss': float(test_loss) if test_loss is not None else None,

    # correlations
    'pearson_r': float(g['pearson_r']),
    'spearman_r': float(g['spearman_r']),
    'r2': float(g['r2']),

    # tolerance accuracies
    'acc_tol_0.10': float(g['acc_01']),
    'acc_tol_0.20': float(g['acc_02']),
    'acc_tol_0.10_clean': float(g['acc_01_clean']) if ('acc_01_clean' in g and g['acc_01_clean'] is not None) else None,
    'acc_tol_0.20_clean': float(g['acc_02_clean']) if ('acc_02_clean' in g and g['acc_02_clean'] is not None) else None,
    'acc_tol_0.10_adv': float(g['acc_01_adv']) if ('acc_01_adv' in g and g['acc_01_adv'] is not None) else None,
    'acc_tol_0.20_adv': float(g['acc_02_adv']) if ('acc_02_adv' in g and g['acc_02_adv'] is not None) else None,

    # AUROC
    'auroc': float(g['roc_auc']),

    # binary classification
    'threshold': float(g['threshold']),
    'binary_acc': float(g['acc']),
    'precision': float(g['precision']),
    'recall': float(g['recall']),
    'f1': float(g['f1']),
    'tp': int(g['tp']),
    'fp': int(g['fp']),
    'fn': int(g['fn']),
    'tn': int(g['tn']),
}

# Backup existing CSV as a copy (timestamped) before writing
# backup_path = None
# if csv_path.exists():
#     backup_path = results_dir / f'metrics_copy_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
#     shutil.copy2(csv_path, backup_path)

# # Create new or append to existing, preserving all columns across runs
# new_row_df = pd.DataFrame([row])
# if csv_path.exists():
#     existing_df = pd.read_csv(csv_path)
#     # Union columns to avoid dropping any
#     full_cols = list(dict.fromkeys(list(existing_df.columns) + list(new_row_df.columns)))
#     existing_df = existing_df.reindex(columns=full_cols)
#     new_row_df = new_row_df.reindex(columns=full_cols)
#     out_df = pd.concat([existing_df, new_row_df], ignore_index=True)
# else:
#     out_df = new_row_df

# out_df.to_csv(csv_path, index=False)
# print(f"Saved metrics to {csv_path}")
# if backup_path is not None:
#     print(f"Backup created at {backup_path}")


# Backup existing CSV as a copy (timestamped) before writing
backup_path = None
if csv_path.exists():
    backup_path = results_dir / f'metrics_copy_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    shutil.copy2(csv_path, backup_path)

# Create new or replace existing row if match found, preserving all columns across runs
new_row_df = pd.DataFrame([row])
if csv_path.exists():
    existing_df = pd.read_csv(csv_path)
    # Union columns to avoid dropping any
    full_cols = list(dict.fromkeys(list(existing_df.columns) + list(new_row_df.columns)))
    existing_df = existing_df.reindex(columns=full_cols)
    new_row_df = new_row_df.reindex(columns=full_cols)
    
    # Columns to match on for replacement
    match_cols = ['model_class', 'width', 'depth', 'lr', 'weight_decay', 
                  'batch_size', 'epochs', 'n_samples', 'target_class']
    
    # Check if a matching row exists
    match_mask = None
    for col in match_cols:
        if col in existing_df.columns and col in new_row_df.columns:
            # Get the new value
            new_val = new_row_df[col].iloc[0]
            
            # Compare column values, handling NaN properly
            if pd.isna(new_val):
                col_match = existing_df[col].isna()
            else:
                col_match = (existing_df[col] == new_val)
            
            # Initialize or combine with previous matches
            if match_mask is None:
                match_mask = col_match
            else:
                match_mask = match_mask & col_match
        else:
            # If column missing, no match possible
            match_mask = pd.Series([False] * len(existing_df))
            break
    
    if match_mask is not None and match_mask.any():
        # Replace the matching row(s) - take first match if multiple
        match_idx = match_mask.idxmax()
        existing_df.loc[match_idx] = new_row_df.iloc[0]
        out_df = existing_df
        print(f"Replaced existing row at index {match_idx}")
    else:
        # Append new row if no match found
        out_df = pd.concat([existing_df, new_row_df], ignore_index=True)
        print("Appended new row (no matching row found)")
else:
    out_df = new_row_df
    print("Created new CSV file")

out_df.to_csv(csv_path, index=False)
print(f"Saved metrics to {csv_path}")
if backup_path is not None:
    print(f"Backup created at {backup_path}")

## inspect metrics file

In [212]:
import pandas as pd

# Load the metrics file
metrics_path = '/rds/general/user/nk1924/home/NADF/data/results/metrics.csv'
metrics_df = pd.read_csv(metrics_path)



In [213]:
metrics_df.columns

In [214]:
metrics_df = metrics_df[metrics_df["acc_tol_0.20"] != 1.0]
metrics_df

In [215]:
# for each target class print row with max acc_tol_0.20
metrics_df[metrics_df["target_class"] == -1].sort_values(by="acc_tol_0.20", ascending=False).head(1)[["acc_tol_0.20", "loss_function", "upweight", "width", "depth", "pearson_r", "acc_tol_0.10", "acc_tol_0.20", "acc_tol_0.10_clean", "acc_tol_0.20_clean", "acc_tol_0.10_adv", "acc_tol_0.20_adv"    ]]

In [216]:
metrics_df[metrics_df["target_class"] == 0].sort_values(by="acc_tol_0.20", ascending=False).head(1)[["acc_tol_0.20", "loss_function", "upweight", "width", "depth", "pearson_r", "acc_tol_0.10", "acc_tol_0.20", "acc_tol_0.10_clean", "acc_tol_0.20_clean", "acc_tol_0.10_adv", "acc_tol_0.20_adv"    ]]

In [217]:
metrics_df[metrics_df["target_class"] == 1].sort_values(by="acc_tol_0.20", ascending=False).head(1)[["acc_tol_0.20", "loss_function", "upweight", "width", "depth", "pearson_r", "acc_tol_0.10", "acc_tol_0.20", "acc_tol_0.10_clean", "acc_tol_0.20_clean", "acc_tol_0.10_adv", "acc_tol_0.20_adv"    ]]

In [218]:
metrics_df[metrics_df["target_class"] == 9].sort_values(by="acc_tol_0.20", ascending=False).head(1)[["acc_tol_0.20", "loss_function", "upweight", "width", "depth", "pearson_r", "acc_tol_0.10", "acc_tol_0.20", "acc_tol_0.10_clean", "acc_tol_0.20_clean", "acc_tol_0.10_adv", "acc_tol_0.20_adv"    ]]

In [ ]:
metrics_df[["loss_function", "upweight", "width", "depth", "target_class", "pearson_r", "acc_tol_0.10", "acc_tol_0.20", "acc_tol_0.10_clean", "acc_tol_0.20_clean", "acc_tol_0.10_adv", "acc_tol_0.20_adv", "roc_auc"]]

In [220]:
# delete rows with acc_tol_0.20== 0
metrics_df = metrics_df[metrics_df["acc_tol_0.20"] != 0]


In [221]:
metrics_df[metrics_df["target_class"] == -1]